In [2]:
import pandas as pd
import numpy as np
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies = movies.merge(credits, on = 'title')

In [6]:
movies = movies[['id', 'title', 'genres', 'keywords', 'cast', 'crew','overview']]
movies.head()

,id,title,genres,keywords,cast,crew,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca..."


In [8]:
movies.dropna(inplace = True)

In [10]:
import ast
def convert(obj):
    l =[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [12]:
movies['genres'] = movies['genres'].apply(convert)

In [14]:
movies['keywords'] = movies['keywords'].apply(convert)

In [15]:
def convert2(obj):
    l =[]
    count=0
    for i in ast.literal_eval(obj):
        if count !=3:
            l.append(i['name'])
            count+=1
        else:
            break
    return l

In [16]:
movies['cast'] = movies['cast'].apply(convert2)

In [19]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == "Director":
            l.append(i['name'])
            break
    return l

In [20]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [21]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [22]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [23]:
movies['tags'] =movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [24]:
new_df = movies[['id', 'title', 'tags']]
new_df.head()

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [25]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

C:\Users\hp\AppData\Local\Temp\ipykernel_21220\2779914785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))
C:\Users\hp\AppData\Local\Temp\ipykernel_21220\2779914785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [28]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [29]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [32]:
import nltk 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [33]:
def stem(text):
    y = []
    for i in text.split():
        ps.stem(i)
    return " ".join(y)

In [34]:
new_df["tags"] = new_df["tags"].apply(stem)

C:\Users\hp\AppData\Local\Temp\ipykernel_21220\825579503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(stem)


In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
similarity = cosine_similarity(vector) 

In [37]:
def recommendation(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    dist = similarity[movie_index]
    movies_list = sorted(list(enumerate(dist)), reverse = True, key = lambda x: x[1])
    movies_list = movies_list[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [38]:
recommendation('Batman')

Batman
Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Returns


In [39]:
import pickle

In [40]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [41]:
pickle.dump(new_df.to_dict,open('movies_dict.pkl','wb'))

In [48]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [66]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)